In [1]:
# !pip install ollama

In [2]:
import json
import random
import requests

import ollama

In [3]:
known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [4]:
get_weather("sao paulo")

31.9

In [5]:
TODO1 = "get_weather"
TODO2 = "Get the current weather for a specified city."
TODO3 = "city"
TODO4 = "The name of the city for which we want to get the temperature"
TODO5 = "city"

get_weather_tool = {
    "type": "function",
    "name": TODO1,
    "description": TODO2, 
    "parameters": {
        "type": "object",
        "properties": {
            TODO3: {
                "type": "string",
                "description": TODO4
            }
        },
        "required": [TODO5],
        "additionalProperties": False
    }
}

In [6]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Add or update weather data for a specified city in our database.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The name of the city for which to set the weather."
            },
            "temp": {
                "type": "number", # Recomenda-se "number" para floats em JSON Schema
                "description": "The temperature value to be associated with the given city."
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}


In [7]:
available_tools = {
    "get_weather": get_weather,
    "set_weather": set_weather
}

In [30]:
# query = "What's the weather in Sao Paulo?"
query = "add the current weather of sao paulo to our database"

response = ollama.chat(
    model="llama3.2:3b",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant specialized in weather information. Use the available tools to answer questions or modify weather data."},
        {"role": "user", "content": query}
    ],
    tools=[get_weather_tool, set_weather_tool],
    options={"num_predict": -1} # Ajuda a garantir que o modelo não pare de gerar antes de decidir a tool_call
)

message = response["message"]

In [31]:
message

Message(role='assistant', content='', thinking=None, images=None, tool_calls=[ToolCall(function=Function(name='getWeather', arguments={'city': 'São Paulo'}))])

In [32]:
message_dict = dict(message)

if "tool_calls" in message_dict.keys() and message_dict["tool_calls"]:
    for tool_call in message_dict["tool_calls"]:
        tool_name = tool_call["function"]["name"]
        tool_args = tool_call["function"]["arguments"]

        print(tool_name, tool_args)

getWeather {'city': 'São Paulo'}


In [34]:
# !pip install fastmcp # https://github.com/jlowin/fastmcp

In [35]:
import fastmcp

In [36]:
fastmcp.__version__

'2.11.0'

In [ ]:
# Starting MCP server 'Demo 🚀' with transport 'stdio'

# Standart input/output means that the client write something to stdin and 
# read the answer using stdout.

In [ ]:
# {"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "get_weather", "arguments": {"city": "berlin"}}}

# result 

# {"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}

In [ ]:
# Available tools: 
# [Tool(name='get_weather', title=None, description='Retrieves the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to retrieve weather data.\n\nReturns:\n    float: The temperature associated with the city.', inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': '_WrappedResult', 'type': 'object', 'x-fastmcp-wrap-result': True}, annotations=None, meta={'_fastmcp': {'tags': []}}), Tool(name='set_weather', title=None, description="Sets the temperature for a specified city.\n\nParameters:\n    city (str): The name of the city for which to set the weather data.\n    temp (float): The temperature to associate with the city.\n\nReturns:\n    str: A confirmation string 'OK' indicating successful update.", inputSchema={'properties': {'city': {'title': 'City', 'type': 'string'}, 'temp': {'title': 'Temp', 'type': 'number'}}, 'required': ['city', 'temp'], 'type': 'object'}, outputSchema=None, annotations=None, meta={'_fastmcp': {'tags': []}})]

In [ ]:
# to be finished
import mcp_client_sync

our_mcp_client = mcp_client_sync.MCPClient(["python", "weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()